# DSPy Multi-Agent System for Intelligent Buying System

This notebook implements a sophisticated multi-agent buying system using DSPy for declarative language model programming, automatic prompt optimization, and systematic evaluation.

## 1. Core DSPy Setup

In [27]:
import dspy
from dspy import Module, ChainOfThought, Predict, ReAct
import json
import pandas as pd
from typing import List, Dict, Any
import asyncio
from datetime import datetime

In [28]:
# Configure DSPy with your LM
# Note: Replace with your actual OpenAI API key or use environment variable
lm = dspy.LM(model="gpt-4", max_tokens=1000)
dspy.settings.configure(lm=lm)

## 2. DSPy Signatures for Agent Tasks

In [29]:
class SupplierEvaluation(dspy.Signature):
    """Evaluate supplier reliability based on historical performance."""
    order_history = dspy.InputField(desc="Past orders from supplier")
    supplier_info = dspy.InputField(desc="Supplier details")
    reliability_score = dspy.OutputField(desc="Score 0-100")
    risk_factors = dspy.OutputField(desc="Key risks identified")

class StockoutPrediction(dspy.Signature):
    """Predict stockout risk for inventory items."""
    current_stock = dspy.InputField()
    consumption_pattern = dspy.InputField()
    lead_time = dspy.InputField()
    stockout_probability = dspy.OutputField()
    days_until_stockout = dspy.OutputField()

class PurchaseRecommendation(dspy.Signature):
    """Generate optimal purchase recommendations."""
    stockout_risk = dspy.InputField()
    supplier_scores = dspy.InputField()
    budget_constraints = dspy.InputField()
    recommended_order = dspy.OutputField()
    justification = dspy.OutputField()

class NegotiationStrategy(dspy.Signature):
    """Generate negotiation strategy for supplier discussions."""
    supplier_profile = dspy.InputField(desc="Supplier characteristics and history")
    order_details = dspy.InputField(desc="Proposed order information")
    market_conditions = dspy.InputField(desc="Current market state")
    negotiation_approach = dspy.OutputField(desc="Recommended negotiation strategy")
    key_leverage_points = dspy.OutputField(desc="Advantages to emphasize")

## 3. DSPy Agent Modules

In [30]:
class DSPySupplierAgent(Module):
    def __init__(self):
        super().__init__()
        self.evaluate = ChainOfThought(SupplierEvaluation)
        
    def forward(self, supplier_data):
        return self.evaluate(
            order_history=supplier_data['history'],
            supplier_info=supplier_data['info']
        )

In [31]:
class DSPyStockoutAgent(Module):
    def __init__(self):
        super().__init__()
        self.predict = Predict(StockoutPrediction)
        
    def forward(self, inventory_data):
        predictions = []
        for sku in inventory_data:
            pred = self.predict(
                current_stock=sku['stock'],
                consumption_pattern=sku['usage'],
                lead_time=sku['lead_time']
            )
            predictions.append(pred)
        return predictions

In [32]:
class DSPyRecommendationAgent(Module):
    def __init__(self):
        super().__init__()
        self.recommend = ChainOfThought(PurchaseRecommendation)
        
    def forward(self, context):
        return self.recommend(**context)

In [33]:
class DSPyNegotiationAgent(Module):
    def __init__(self):
        super().__init__()
        self.strategize = ChainOfThought(NegotiationStrategy)
        
    def forward(self, negotiation_context):
        return self.strategize(
            supplier_profile=negotiation_context['supplier'],
            order_details=negotiation_context['order'],
            market_conditions=negotiation_context['market']
        )

## 4. Multi-Agent Orchestration

In [34]:
class DSPyBuyingSystemOrchestrator(Module):
    def __init__(self):
        super().__init__()
        self.supplier_agent = DSPySupplierAgent()
        self.stockout_agent = DSPyStockoutAgent()
        self.recommendation_agent = DSPyRecommendationAgent()
        self.negotiation_agent = DSPyNegotiationAgent()
        
    def forward(self, system_state):
        # Parallel agent execution
        supplier_scores = self.supplier_agent(system_state['suppliers'])
        stockout_risks = self.stockout_agent(system_state['inventory'])
        
        # Aggregate insights
        context = {
            'stockout_risk': stockout_risks,
            'supplier_scores': supplier_scores,
            'budget_constraints': system_state['budget']
        }
        
        # Generate recommendations
        recommendations = self.recommendation_agent(context)
        
        # Apply business rules validation
        # Note: Manual validation since dspy.Assert is not available in current version
        if '$' in str(recommendations.recommended_order):
            try:
                order_amount = float(recommendations.recommended_order.split('$')[1])
                if order_amount > system_state['budget']:
                    raise ValueError("Order must be within budget")
            except (ValueError, IndexError):
                pass  # Skip validation if parsing fails
        
        # Generate negotiation strategy if needed
        if system_state.get('negotiation_required', False):
            negotiation_strategy = self.negotiation_agent({
                'supplier': system_state['selected_supplier'],
                'order': recommendations.recommended_order,
                'market': system_state.get('market_conditions', {})
            })
            recommendations.negotiation_strategy = negotiation_strategy
        
        return recommendations

## 5. Optimization Strategy

In [35]:
def load_historical_data():
    """Load historical data for training examples."""
    # This would load from your actual data files
    # For demo purposes, we'll create sample data
    sample_data = [
        {
            'system_state': {
                'suppliers': {'history': 'good performance', 'info': 'reliable supplier'},
                'inventory': [{'stock': 100, 'usage': 'high', 'lead_time': '5 days'}],
                'budget': 10000
            },
            'optimal_decision': 'Order 500 units from Supplier A'
        }
    ]
    return [(data['system_state'], data['optimal_decision']) for data in sample_data]

def procurement_success_metric(example, pred, trace=None):
    """Metric to evaluate procurement decisions."""
    # Simple metric - in practice this would be more sophisticated
    return len(str(pred.recommended_order)) > 10  # Basic validation

In [36]:
# Create training examples from historical data
def setup_optimization():
    train_examples = [
        dspy.Example(
            system_state=historical_state,
            optimal_decision=historical_outcome
        ).with_inputs('system_state')
        for historical_state, historical_outcome in load_historical_data()
    ]
    
    # Optimize with DSPy compiler
    from dspy.teleprompt import BootstrapFewShot
    
    optimizer = BootstrapFewShot(metric=procurement_success_metric)
    orchestrator = DSPyBuyingSystemOrchestrator()
    
    try:
        optimized_orchestrator = optimizer.compile(
            orchestrator,
            trainset=train_examples
        )
        return optimized_orchestrator
    except Exception as e:
        print(f"Optimization failed: {e}")
        return orchestrator

## 6. Demo and Testing

In [37]:
# Sample system state for testing
sample_system_state = {
    'suppliers': {
        'history': 'Supplier A: 95% on-time delivery, Supplier B: 87% on-time delivery',
        'info': 'Supplier A: Premium quality, higher cost. Supplier B: Standard quality, competitive pricing'
    },
    'inventory': [
        {
            'sku': 'WIDGET-001',
            'stock': 50,
            'usage': 'Average 20 units per week, trending upward',
            'lead_time': '7 days'
        },
        {
            'sku': 'GADGET-002', 
            'stock': 200,
            'usage': 'Average 15 units per week, stable',
            'lead_time': '14 days'
        }
    ],
    'budget': 25000,
    'negotiation_required': True,
    'selected_supplier': 'Supplier A with premium quality focus',
    'market_conditions': 'Raw material costs increasing, supply chain disruptions possible'
}

In [38]:
# Initialize and run the system
def run_buying_system_demo():
    print("Initializing DSPy Buying System...")
    orchestrator = DSPyBuyingSystemOrchestrator()
    
    print("Running system with sample data...")
    try:
        result = orchestrator(sample_system_state)
        
        print("\n=== BUYING SYSTEM RECOMMENDATIONS ===")
        print(f"Recommended Order: {result.recommended_order}")
        print(f"Justification: {result.justification}")
        
        if hasattr(result, 'negotiation_strategy'):
            print(f"\nNegotiation Strategy: {result.negotiation_strategy.negotiation_approach}")
            print(f"Key Leverage Points: {result.negotiation_strategy.key_leverage_points}")
            
        return result
    except Exception as e:
        print(f"Error running system: {e}")
        return None

# Uncomment to run the demo (requires valid OpenAI API key)
result = run_buying_system_demo()

Initializing DSPy Buying System...
Running system with sample data...

=== BUYING SYSTEM RECOMMENDATIONS ===
Recommended Order: Order the first product from Supplier A and the second product from Supplier B.
Justification: The first product has a high risk of stockout and less than two weeks until stockout, making it critical to ensure on-time delivery. Supplier A, with a higher reliability score, is more likely to deliver on time, reducing the risk of stockout. However, due to budget constraints, it is necessary to also consider cost. Supplier B, while less reliable, offers competitive pricing and could be a more cost-effective choice for the second product, which has a low risk of stockout and more than 90 days until stockout.

Negotiation Strategy: The recommended negotiation strategy is to leverage the order split between Supplier A and B. This can be used to negotiate better terms with Supplier A by suggesting a potential increase in order volume if they can offer competitive pric

## 7. Integration with Existing System

In [24]:
def integrate_with_existing_data():
    """Integration function to load data from existing CSV files and systems."""
    try:
        # Load inventory data
        inventory_df = pd.read_csv('data/inventory.csv')
        
        # Load supplier data  
        suppliers_df = pd.read_csv('data/suppliers.csv')
        
        # Convert to system state format
        system_state = {
            'inventory': inventory_df.to_dict('records'),
            'suppliers': {
                'history': suppliers_df.to_string(),
                'info': f"Available suppliers: {len(suppliers_df)}"
            },
            'budget': 50000  # Default budget
        }
        
        return system_state
    except FileNotFoundError:
        print("Data files not found. Using sample data.")
        return sample_system_state

def save_recommendations(recommendations, filename=None):
    """Save recommendations to file for integration with monitoring systems."""
    if filename is None:
        filename = f"recommendations_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    rec_data = {
        'timestamp': datetime.now().isoformat(),
        'recommended_order': str(recommendations.recommended_order),
        'justification': str(recommendations.justification)
    }
    
    with open(filename, 'w') as f:
        json.dump(rec_data, f, indent=2)
    
    print(f"Recommendations saved to {filename}")

## 8. Monitoring and Evaluation

In [25]:
class SystemMonitor:
    def __init__(self):
        self.metrics = []
        
    def log_decision(self, system_state, recommendations, outcome=None):
        """Log system decisions for analysis and improvement."""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'system_state': system_state,
            'recommendations': str(recommendations.recommended_order),
            'justification': str(recommendations.justification),
            'outcome': outcome
        }
        self.metrics.append(log_entry)
        
    def evaluate_performance(self):
        """Evaluate system performance over time."""
        if not self.metrics:
            return "No data to evaluate"
            
        successful_decisions = sum(1 for m in self.metrics if m.get('outcome') == 'success')
        total_decisions = len(self.metrics)
        
        return {
            'total_decisions': total_decisions,
            'successful_decisions': successful_decisions,
            'success_rate': successful_decisions / total_decisions if total_decisions > 0 else 0
        }
        
    def export_metrics(self, filename='system_metrics.json'):
        """Export metrics for external analysis."""
        with open(filename, 'w') as f:
            json.dump(self.metrics, f, indent=2)
        print(f"Metrics exported to {filename}")

# Initialize monitor
monitor = SystemMonitor()

## 9. Next Steps and Usage

This DSPy multi-agent system provides:

1. **Structured Decision Making**: Clear signatures define inputs/outputs for each agent
2. **Automatic Optimization**: DSPy can improve performance through training
3. **Explainable AI**: Reasoning chains show how decisions are made
4. **Modular Architecture**: Easy to extend with new agents or modify existing ones
5. **Integration Ready**: Connects with existing data pipelines and monitoring

To use this system:
1. Set up your OpenAI API key
2. Load your actual inventory and supplier data
3. Run the orchestrator with real system state
4. Monitor and collect feedback for optimization
5. Use DSPy's compilation to improve performance over time